---
title: 02 - human tunning
author: Martin Proks
date: 07-12-2023
---

This notebook contains multiple methods on how we trained the model. We summarize below which params were helpful in generating better integration.

- `n_layers`: should be 2 - 3
- `gene_dispersion`: `gene` proved to be the best
- `gene_likelihood`: `nb` prefered over `zinb`
- `dropout_rate`: smaller penalization keeps datapoints closer (`0.005`)

In [1]:
!which pip

~/projects/data/Brickman/conda/envs/scvi-1.0.0/bin/pip


In [2]:
import scvi
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt


from rich import print
from scib_metrics.benchmark import Benchmarker
from scvi.model.utils import mde


import warnings
from lightning_fabric.plugins.environments.slurm import PossibleUserWarning
warnings.simplefilter(action='ignore', category=PossibleUserWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

scvi.settings.seed = 42

/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[rank: 0] Global seed set to 42


In [3]:
sc.set_figure_params(figsize=(10, 6))

%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

plt.rcParams['svg.fonttype'] = 'none'

In [4]:
adata = sc.read("../data/processed/32_human_adata.h5ad")
adata

AnnData object with n_obs × n_vars = 2323 × 62754
    obs: 'day', 'ct', 'experiment', 'technology', 'n_counts', 'n_genes', 'ct_fine'
    layers: 'counts'

In [5]:
adata.obs.experiment.unique().tolist()

['Meistermann_2021',
 'Petropoulos_2016',
 'Xiang_2020',
 'Yan_2013',
 'Yanagida_2021',
 'Xue_2013']

In [7]:
sc.pp.highly_variable_genes(
    adata,
    flavor="seurat_v3",
    n_top_genes=3_000,
    layer="counts",
    batch_key="experiment",
    subset=True,
)

## 3. Pimp my model: ray tunner

In [8]:
import ray
import jax
import os

from ray import tune
from scvi import autotune

os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

In [9]:
jax.devices()

[gpu(id=0), gpu(id=1)]

In [11]:
ref_tuner = sc.AnnData(adata.layers["counts"])
ref_tuner.obs = adata.obs[["technology", "experiment"]].copy()

model_cls = scvi.model.SCVI
model_cls.setup_anndata(ref_tuner, 
                        batch_key="experiment")

scvi_tuner = autotune.ModelTuner(model_cls)

In [12]:
scvi_tuner.info()

ModelTuner registry for SCVI

                  Tunable hyperparameters                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃      Hyperparameter      ┃ Default value ┃    Source    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│         n_hidden         │      128      │     VAE      │
│         n_latent         │      10       │     VAE      │
│         n_layers         │       1       │     VAE      │
│       dropout_rate       │      0.1      │     VAE      │
│        dispersion        │     gene      │     VAE      │
│     gene_likelihood      │     zinb      │     VAE      │
│   latent_distribution    │    normal     │     VAE      │
│    encode_covariates     │     False     │     VAE      │
│ deeply_inject_covariates │     True      │     VAE      │
│      use_batch_norm      │     both      │     VAE      │
│      use_layer_norm      │     none      │     VAE      │
│        optimizer         │     Adam      │ TrainingPlan │
│            lr            │     0.001     │ TrainingPlan │
│       weight_decay       │     1e-06     │ TrainingPlan │
│           eps            │     0.01      │ TrainingPlan │
│    n_steps_kl_warmup     │     None      │ TrainingPlan │
│    n_epochs_kl_warmup    │      400      │ TrainingPlan │
│   reduce_lr_on_plateau   │     False     │ TrainingPlan │
│        lr_factor         │      0.6      │ TrainingPlan │
│       lr_patience        │      30       │ TrainingPlan │
│       lr_threshold       │      0.0      │ TrainingPlan │
│          lr_min          │       0       │ TrainingPlan │
│      max_kl_weight       │      1.0      │ TrainingPlan │
│      min_kl_weight       │      0.0      │ TrainingPlan │
└──────────────────────────┴───────────────┴──────────────┘

       Available metrics        
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃     Metric      ┃    Mode    ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ validation_loss │    min     │
└─────────────────┴────────────┘

                         Default search space                         
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Hyperparameter ┃ Sample function ┃  Arguments  ┃ Keyword arguments ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│    n_hidden    │     choice      │ [[64, 128]] │        {}         │
└────────────────┴─────────────────┴─────────────┴───────────────────┘

In [13]:
search_space = {
    "gene_likelihood": tune.choice(["nb", "zinb"]),
    "dispersion": tune.choice(["gene", "gene-batch"]),
    "n_hidden": tune.choice([128, 144, 256]),
    "n_layers": tune.choice([2, 3, 4, 5]),
    "lr": tune.loguniform(1e-4, 0.6),
}

In [14]:
ray.init(
    log_to_driver=False,
    num_cpus=10,
    num_gpus=2,
)

2023-12-07 12:27:30,713	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.10.11
Ray version:,2.5.1


In [15]:
results = scvi_tuner.fit(
    ref_tuner,
    metric="validation_loss",
    search_space=search_space,
    num_samples=50,
    max_epochs=100,
)

/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1559: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a,
2023-12-07 12:33:20,350	ERROR tune_controller.py:873 -- Trial task failed for trial _trainable_f3b1a06f
Traceback (most recent call last):
  File "/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 18, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/fdb589/projects/data/Brickman/conda/envs/scvi-1.0.0/lib/python3.10/site-package

In [16]:
print(results.model_kwargs)
print(results.train_kwargs)
print(results.metric)

{'gene_likelihood': 'zinb', 'dispersion': 'gene', 'n_hidden': 144, 'n_layers': 2}

{'plan_kwargs': {'lr': 0.0026974298754178687}}

{'metric': 'validation_loss', 'mode': 'min', 'value': 9358.837890625}

In [17]:
import pandas as pd


training = pd.DataFrame([
    [x.metrics['validation_loss']] + x.path.split(',')[1:]
    for x in results.results if 'validation_loss' in x.metrics
]).sort_values(by=0)

training.to_csv("../results/02_human_integration/tunning.csv")
display(training.head(10))

,0,1,2,3,4
9,9358.837891,gene_likelihood=zinb,lr=0.0027,n_hidden=144,n_layers=2_2023-12-07_12-37-28
6,10226.864258,gene_likelihood=zinb,lr=0.0244,n_hidden=128,n_layers=4_2023-12-07_12-33-17
1,10236.216797,gene_likelihood=zinb,lr=0.0018,n_hidden=256,n_layers=5_2023-12-07_12-27-47
2,10415.340820,gene_likelihood=zinb,lr=0.0016,n_hidden=144,n_layers=2_2023-12-07_12-30-03
0,11194.058594,gene_likelihood=zinb,lr=0.0366,n_hidden=128,n_layers=2_2023-12-07_12-27-37
5,13192.603516,gene_likelihood=nb,lr=0.0211,n_hidden=144,n_layers=4_2023-12-07_12-32-37
12,13270.201172,gene_likelihood=zinb,lr=0.0010,n_hidden=128,n_layers=4_2023-12-07_12-38-25
10,14023.433594,gene_likelihood=zinb,lr=0.0006,n_hidden=144,n_layers=3_2023-12-07_12-37-32
4,14757.439453,gene_likelihood=zinb,lr=0.0001,n_hidden=128,n_layers=2_2023-12-07_12-32-30
15,20656.691406,gene_likelihood=nb,lr=0.0059,n_hidden=144,n_layers=5_2023-12-07_12-40-10
